# Driving a Financial Valuation from a Design

As seen in
[Loading a Design](../load_design.ipynb), a design's object model (a
[knowledge graph](https://en.wikipedia.org/wiki/Knowledge_graph) including all
its spatial and semantic relationships) can be loaded via Speckle from any of
Speckle's supported [connectors](https://speckle.guide/user/connectors.html), so
long as those objects (`Entity`s and `Relationship`s) have been assigned.

In this notebook, we will demonstrate how to integrate financial modelling into
the object model (graph) in order to derive the financial valuation of a real
estate property from its spatial design.
This is done by calculating cash flows per `Entity`, and then using the
`Relationship`s to aggregate those `Flow`s into appropriate `Stream`s.

## Load the Design
We will use the same design as in the previous notebook.

In [1]:
import json
import os

import locale
import urllib.parse
import json
import pandas as pd
import networkx as nx
from IPython.display import IFrame
import plotly.subplots
import plotly.offline as py

import rangekeeper as rk

In [2]:
speckle = rk.api.Speckle(
    host="speckle.xyz",
    token=os.getenv('SPECKLE_TOKEN'))
stream_id = "f5e306e3fa"
commit_id = speckle.get_latest_commit_id(stream_id)
IFrame("https://speckle.xyz/embed?stream={0}&commit={1}".format(stream_id, commit_id), width='100%', height=800)


 SpeckleClient( server: https://speckle.xyz, authenticated: True )


### Inspect the Model Graph

In [3]:
model = speckle.get_commit(stream_id=stream_id)
parsed = rk.api.Speckle.parse(base=model['@property'])
property = rk.api.Speckle.to_rk(
    bases=list(parsed.values()),
    name='property',
    type='archetype')

Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.


In [4]:
property.plot(name=f'assets/{property.name}')
IFrame(src=f'./{property.name}.html', width='100%', height=800)

assets/property.html


## Assigning and Aggregating `Flow`s per `Entity`
Now we can organise the two main categories of cash flows (costs and revenues)
according to the `Relationship` types in the design.

In this example, revenue-based cash flows are generated by `Entity`s that have a
measureable floor area -- in this case, anything that has a 'gfa' property.

We can query the graph to find them:

### Revenue-producing Entities
Revenue-producing `Entity`s will be aggregated by their spatial containment. So
we can first slice our graph to a tree of nodes that have "contains"
relationships:

In [5]:
spatial_containment = property.filter_by_type(
    relationship_type='spatiallyContains',
    name='spatial_containment')

Note: we anticipate that the spatial decomposition is completely hierarchical
(i.e., no spaces overlap, which means no spaces have multiple parents). We can
check this by testing whether the containment graph is a "tree" (or
"arborescence": see [NetworkX Tree](https://networkx.org/documentation/stable/reference/algorithms/tree.html#tree)).

In [6]:
nx.is_arborescence(spatial_containment.graph)

True

In [7]:
# Plot the spatial containment graph
spatial_containment.plot(name=f'assets/{spatial_containment.name}')
IFrame(src=f'./{spatial_containment.name}.html', width='100%', height=800)

assets/property by spatiallyContains and None.html


#### Simple Aggregation
Now that we have verified this, we can continue with a simple aggregation. We
wish to aggregate up through our hierarchy all `Entity`s that have a Gross Floor
Area ('gfa') property. *Rangekeeper* provides defaults for the `Entity.aggregate()`
method for this, whereby a named property of any `Entity` is summed (numerical
addition) into its parent `Entity` against a specified label:

In [8]:
property.aggregate(
    property='gfa',
    label='subtotal_gfa',
    relationship_type='spatiallyContains')

We can tabulate this by converting the graph to a pandas `DataFrame`. Note that
the graph nodes (as witnessed in the DataFrame's index and 'parent' column)
are GUID strings. Each of these have corresponding `Entity` objects, organised
via NetworkX's Node Attributes. See [NetworkX Tutorial](https://networkx.org/documentation/stable/tutorial.html#what-to-use-as-nodes-and-edges)

In [9]:
df = spatial_containment.to_DataFrame()
df = df[['name', 'type', 'gfa', 'subtotal_gfa', 'parent', 'use', 'ffl', 'number']]
df

,name,type,gfa,subtotal_gfa,parent,use,ffl,number
4fd8cf16-d01f-4d1e-964b-87942232e857,property,property,NaN,45816.419959,None,NaN,NaN,NaN
b72f69fc-5c1e-4501-99fb-01c3d3b4dddd,buildingB,building,NaN,13436.884829,4fd8cf16-d01f-4d1e-964b-87942232e857,NaN,NaN,NaN
b7673e43-2e24-42df-bc0a-a364891daa94,plinth,building,NaN,12773.744211,4fd8cf16-d01f-4d1e-964b-87942232e857,NaN,NaN,NaN
37e4f7dd-a27d-4e7e-b7f2-3da8bc46b4ac,buildingA,building,NaN,19605.790919,4fd8cf16-d01f-4d1e-964b-87942232e857,NaN,NaN,NaN
d68eab6c-7031-4780-b8da-a356d629dbe4,utilities,utilities,NaN,0.000000,4fd8cf16-d01f-4d1e-964b-87942232e857,NaN,NaN,NaN
62e63c92-aff1-43da-9ff8-538567f943e6,buildingBresidential,space,NaN,11442.736624,b72f69fc-5c1e-4501-99fb-01c3d3b4dddd,residential,NaN,NaN
ae3ed380-4db9-4e55-94c3-259f913205a2,buildingBparking,space,NaN,1178.164752,b72f69fc-5c1e-4501-99fb-01c3d3b4dddd,parking,NaN,NaN
d0dd5d16-1a60-4cf4-9261-873a76b551ae,buildingBretail,space,NaN,815.983454,b72f69fc-5c1e-4501-99fb-01c3d3b4dddd,retail,NaN,NaN
b7db88ba-b8cc-4f29-a67c-b996707a1340,buildingBcores,utilities,NaN,0.000000,b72f69fc-5c1e-4501-99fb-01c3d3b4dddd,cores,NaN,NaN
a7da795b-96a8-4f81-bb89-fda0e88a783c,buildingBresidentialFloor0,floor,765.941631,765.941631,62e63c92-aff1-43da-9ff8-538567f943e6,NaN,0.3,0.0


We can also plot this as a hierarchical pie chart (a.k.a. 'sunburst' chart), or
a treemap:

In [10]:
sunburst = spatial_containment.sunburst(property='subtotal_gfa')
treemap = spatial_containment.treemap(property='subtotal_gfa')
fig = plotly.subplots.make_subplots(
    rows=2, cols=1,
    specs=[[{"type": "sunburst"}], [{"type": "treemap"}]],
    subplot_titles=('Gross Floor Area - Sunburst Plot', 'Gross Floor Area - Treemap Plot'))
fig.append_trace(sunburst, row=1, col=1)
fig.append_trace(treemap, row=2, col=1)
filename='spatial_containment_chart.html'

py.plot(fig, filename=f'assets/{filename}', auto_open=False)
IFrame(src=f'./{filename}', width='100%', height=800 * len(fig.data))

#### Aggregating a (Financial) Calculation
A more complex aggregation involves collecting specific cash `Flow`s into
`Stream`s per `Entity`/`Assembly`; for instance, we may want to aggregate all
revenues generated by lettable floorspace into their respective parent `Entity`s,
in order to analyse or compare the performance of the design at different
resolutions.

We start by identifying which `Entity`s are 'floor' types and have an area
measurement:

In [11]:
floors = property.filter_by_type(entity_type='floor')
floors = [floor for floor in floors.get_entities().values() if hasattr(floor, 'gfa')]

For each floor we can project a set of simple cash flows, in a similar manner
to [A Basic DCF Valuation](../basic_dcf.ipynb).

First initialize our locale, and a set of parameters, including some around
area efficiency ratios, initial incomes per area, and growth rates:

In [12]:
locale.setlocale(locale.LC_ALL, 'en_au')
units = rk.measure.Index.registry
currency = rk.measure.register_currency(registry=units)
period_type = rk.periodicity.Type.YEAR

In [13]:
efficiency_ratios = dict(
    office=0.825,
    retail=0.675,
    residential=0.75,
    parking=0.9
    )
initial_income_per_area_pa = dict(
    office=750,
    retail=1000,
    residential=600,
    parking=0
    )
pgi_growth_rates = dict(
    office=.025,
    retail=.075,
    residential=.055,
    parking=0
    )
vacancy_rates = dict(
    office=.075,
    retail=.5,
    residential=.02,
    parking=0,
    )

In [14]:
revenue_inputs = pd.DataFrame(
    data=dict(
        efficiency_ratio=efficiency_ratios,
        initial_income_per_area_pa=initial_income_per_area_pa,
        pgi_growth_rate=pgi_growth_rates,
        vacancy_rate=vacancy_rates,
        ))
revenue_inputs

,efficiency_ratio,initial_income_per_area_pa,pgi_growth_rate,vacancy_rate
office,0.825,750,0.025,0.075
retail,0.675,1000,0.075,0.500
residential,0.750,600,0.055,0.020
parking,0.900,0,0.000,0.000


In [15]:
params = dict(
    start_date=pd.Timestamp('2001-01-01'),
    num_periods=10,
    period_type=rk.periodicity.Type.YEAR
    )

##### `Stream`s of Revenue `Flow`s per `Entity`:
We can now construct a set of cash flows per `Entity` in our design for our
asset. We will use a similar structure to the previous notebooks.

First we will define a general 'Spans' class that defines the time periods. This
can be used by any subsequent calculations:

In [16]:
class Spans:
    def __init__(self, params: dict):
        self.params = params
        self.calc_span = rk.span.Span.from_num_periods(
            name='Span to Calculate Reversion',
            date=self.params['start_date'],
            period_type=self.params['period_type'],
            num_periods=self.params['num_periods'] + 1)
        self.acq_span = rk.span.Span.from_num_periods(
            name='Acquisition Span',
            date=rk.periodicity.offset_date(
                self.params['start_date'],
                num_periods=-1,
                period_type=self.params['period_type']),
            period_type=self.params['period_type'],
            num_periods=1)
        self.span = self.calc_span.shift(
            name='Span',
            num_periods=-1,
            period_type=self.params['period_type'],
            bound='end')

Then, we will define a `Revenues` class that holds the parameters:

In [17]:
class Revenues:
    def __init__(
            self,
            params: dict,
            spans: Spans):
        self.params = params
        self.spans = spans

And then we add Revenue `Flow`s:

In [18]:
@rk.update_class(Revenues)
class Revenues:
    def generate(self):
        self.pgi = rk.flux.Flow.from_projection(
            name='Potential Gross Income',
            value=self.params['initial_income'],
            proj=rk.projection.Extrapolation(
            form=rk.extrapolation.Compounding(
                rate=self.params['pgi_growth_rate']),
            sequence=self.spans.calc_span.to_index(period_type=self.params['period_type'])),
            units=currency.units)
        self.vacancy = rk.flux.Flow(
            name='Vacancy Allowance',
            movements=self.pgi.movements * -self.params['vacancy_rate'],
            units=currency.units)
        self.egi = rk.flux.Stream(
            name='Effective Gross Income',
            flows=[self.pgi, self.vacancy],
            period_type=self.params['period_type'])

Now we need to add `Entity`-specific parameters to the parameters dictionary,
and run  the `init_spans()` and `init_flows()` methods per floor `Entity`:

(Note we place the resultant `Stream` in the `events` attribute of the `Entity`)

In [19]:
spans = Spans(params)

In [20]:
for floor in floors:
    parent = floor.get_relatives(
        relationship_type='spatiallyContains',
        outgoing=False,
        assembly=spatial_containment)[0]
    floor['use'] = parent['use']

    floor.params = params.copy()
    floor.params['initial_income'] = initial_income_per_area_pa[floor['use']] * floor['gfa'] * efficiency_ratios[floor['use']]
    floor.params['pgi_growth_rate'] = pgi_growth_rates[floor['use']]
    floor.params['vacancy_rate'] = vacancy_rates[floor['use']]

    revenues = Revenues(floor.params, spans)
    revenues.generate()
    floor['pgi'] = revenues.pgi
    floor['vacancy'] = revenues.vacancy
    floor['egi'] = revenues.egi

##### Aggregating by Parent Container
Since we have the "containment" hierarchy, we can aggregate the `Revenues` per
parent `Entity`:

*Rangekeeper* provides some helper methods to assist in the aggregation of
`Stream`s and `Flow`s; specifically, since the `Entity.aggregate()` method
updates a specified `Entity`'s property with a dictionary of subentity
properties, keyed by their respective `entityId`s, we can use the
`aggregate_flows()` function to support the collation of child `Entity`
`Flow`s into a single `Stream` in a new property label in the `Entity`

In [21]:
def aggregate_egis(**kwargs):
    return rk.graph.Entity.aggregate_flows(
        **kwargs,
        name='Effective Gross Income',
        period_type=period_type)

This function is passed into the `Entity.aggregate()` method as the `function`
parameter:

In [22]:
property.aggregate(
    function=aggregate_egis,
    property='egi',
    label='subtotal_egi',
    relationship_type='spatiallyContains')

Now we can look at what the `aggregate` method has produced for a single floor:

In [23]:
floors[1]['egi'].name

'Effective Gross Income'

We can then inspect the `aggregate_egi` property of an `Entity`, which holds a
`Stream` of `Flow`s (each coming from a child of that `Entity`). Let's use the
office compartment in Building A:

In [24]:
buildingAoffice = [e for (id, e) in spatial_containment.get_entities().items() if e.name == 'buildingAoffice'][0]
print(buildingAoffice.name)
buildingAoffice['subtotal_egi']

buildingAoffice


date,Effective Gross Income for b8992cc5-7cef-40a1-9b84-1b0bf7b33dcb [buildingAofficeFloor1],Effective Gross Income for db13e799-4efd-481c-ae1d-db47b6a7e997 [buildingAofficeFloor2],Effective Gross Income for 7fd89991-3575-4f24-93c6-36a15deef8c1 [buildingAofficeFloor3],Effective Gross Income for 3d738ff6-0132-407b-a157-bcdbddab3185 [buildingAofficeFloor4],Effective Gross Income for a47cb204-0317-46bf-a023-000272671b88 [buildingAofficeFloor5],Effective Gross Income for c1c1de38-f85c-4ea5-8a3c-c1b0809419a6 [buildingAofficeFloor6],Effective Gross Income for a1d87916-6fe5-4199-b126-b0d576f80c48 [buildingAofficeFloor7],Effective Gross Income for ab0e1b85-def5-48b5-82d9-8f72f93eb0b3 [buildingAofficeFloor8],Effective Gross Income for 82aa4837-1b06-4df1-be65-c1f644d79704 [buildingAofficeFloor9],Effective Gross Income for 510e17fc-4933-4ec5-be97-a2a2b7f952b6 [buildingAofficeFloor10]
2001,"$944,849.16","$829,281.46","$829,281.46","$829,281.46","$829,281.46","$829,281.46","$829,281.46","$829,281.46","$829,281.46","$829,281.46"
2002,"$968,470.39","$850,013.49","$850,013.49","$850,013.49","$850,013.49","$850,013.49","$850,013.49","$850,013.49","$850,013.49","$850,013.49"
2003,"$992,682.15","$871,263.83","$871,263.83","$871,263.83","$871,263.83","$871,263.83","$871,263.83","$871,263.83","$871,263.83","$871,263.83"
2004,"$1,017,499.20","$893,045.43","$893,045.43","$893,045.43","$893,045.43","$893,045.43","$893,045.43","$893,045.43","$893,045.43","$893,045.43"
2005,"$1,042,936.68","$915,371.56","$915,371.56","$915,371.56","$915,371.56","$915,371.56","$915,371.56","$915,371.56","$915,371.56","$915,371.56"
2006,"$1,069,010.10","$938,255.85","$938,255.85","$938,255.85","$938,255.85","$938,255.85","$938,255.85","$938,255.85","$938,255.85","$938,255.85"
2007,"$1,095,735.35","$961,712.25","$961,712.25","$961,712.25","$961,712.25","$961,712.25","$961,712.25","$961,712.25","$961,712.25","$961,712.25"
2008,"$1,123,128.74","$985,755.05","$985,755.05","$985,755.05","$985,755.05","$985,755.05","$985,755.05","$985,755.05","$985,755.05","$985,755.05"
2009,"$1,151,206.95","$1,010,398.93","$1,010,398.93","$1,010,398.93","$1,010,398.93","$1,010,398.93","$1,010,398.93","$1,010,398.93","$1,010,398.93","$1,010,398.93"
2010,"$1,179,987.13","$1,035,658.90","$1,035,658.90","$1,035,658.90","$1,035,658.90","$1,035,658.90","$1,035,658.90","$1,035,658.90","$1,035,658.90","$1,035,658.90"


We can also easily sum and collapse that aggregation, as it is a Stream`:

In [25]:
buildingAoffice['subtotal_egi'].sum()

date,Effective Gross Income for f746d0de-a8ab-491e-9d3f-875c62c87f53 [buildingAoffice] Aggregation (sum)
2001-12-31 00:00:00,"$8,408,382.26"
2002-12-31 00:00:00,"$8,618,591.82"
2003-12-31 00:00:00,"$8,834,056.61"
2004-12-31 00:00:00,"$9,054,908.03"
2005-12-31 00:00:00,"$9,281,280.73"
2006-12-31 00:00:00,"$9,513,312.75"
2007-12-31 00:00:00,"$9,751,145.57"
2008-12-31 00:00:00,"$9,994,924.20"
2009-12-31 00:00:00,"$10,244,797.31"
2010-12-31 00:00:00,"$10,500,917.24"


In [26]:
buildingAoffice['subtotal_egi'].sum().collapse()

date,Effective Gross Income for f746d0de-a8ab-491e-9d3f-875c62c87f53 [buildingAoffice] Aggregation (sum)
2011-12-31 00:00:00,"$104,965,756.69"


The above subtotals the Effective Gross Income (EGI) for the office space of
Building A over the hold period of the asset.

We can then do this for all `Entity`s in the graph, and plot the results:

In [27]:
sunburst = spatial_containment.sunburst(property='subtotal_egi')
treemap = spatial_containment.treemap(property='subtotal_egi')
fig = plotly.subplots.make_subplots(
    rows=2, cols=1,
    specs=[[{"type": "sunburst"}], [{"type": "treemap"}]],
    subplot_titles=('Effective Gross Income - Sunburst Plot', 'Effective Gross Income - Treemap Plot'))
fig.append_trace(sunburst, row=1, col=1)
fig.append_trace(treemap, row=2, col=1)
filename='aggregate_egis_chart.html'

py.plot(fig, filename=f'assets/{filename}', auto_open=False)
IFrame(src=f'./{filename}', width='100%', height=800 * len(fig.data))

### Cost-generating Entities
Let's now do the same for the cost-generating `Entity`s. While it is routine to
register revenue-generating `Flow`s with `Entity`s that have lettable or
sellable floor space, the convention to also register all cost-generating
`Flow`s with only those same `Entity`s is possibly only done out of convenience
rather than accuracy.

For example, both operating and capital expenses may be attributed to building
systems (e.g. maintenance, upgrading, & replacement of MEP or facade
componentry) that are not coincident with the floor space they are servicing.
I.e., the convention to average (sub)total costs across (sub)total floor space
may be misleading, especially when attempting to compare different scenarios at
different resolutions (See {cite}`farevuu2018`, 5.4 Flaw of Averages).

By virtue of *Rangekeeper*'s multi-faceted approach to its object model, costs
can be attributed and aggregated to `Entity`s that are not spatially coincident
with the revenue-generating `Entity`s; in the example design, we will
use the 'services' relationship-type to register and aggregate some of these,
while space-specific costs will be registered and aggregated via spatial
containment:

#### Operational and Capital Expenses of Building Utilities & Systems
Of course, in a simplified model and without knowing design and engineering
details for the property, costs for operational and capital expenses will be
calculated from some proportional rate of spatial characteristics of the
building. In this example, we will use the following as simplifications of how
these types of costs may be calculated. Note that this is more a demonstration
of methodology, and in practice would adjust to real-world data about the
specific building systems and their costs.
-  'Floorplate-derived' Operational and Capital Expenses will be calculated as a
function of the subtotal floorplate area of 'building' `Entity`s. One could
think of this as representing the cost of operating the floor's MEP, or
maintaining, repairing and upgrading elements on the floors themselves
(e.g. FF&E)
- 'Facade-derived' Operational Expenses will be calculated as a function of the
  surface area of 'building' `Entity`. This would represent the cost of facade
maintenance, repair or replacement.
- 'Utility-derived' Capital Expenses will be calculated as a function of the
volume of 'utility' `Entity`s. This represents the cost of repairing and
upgrading the building's MEP and transportation systems.


#### Cost `Flow`s per `Entity`
There are some properties of `Entity`s that we need to produce before we can
calculate `Flow`s off them. In this case we need to produce the facade area
for any `Entity`s that have 'perimeter' and 'ftf' (Floor-to-Floor height)
properties:


In [28]:
# Aggregate Facade Areas:
for (entityId, entity) in property.get_entities().items():
    if hasattr(entity, 'perimeter') & hasattr(entity, 'ftf'):
        entity['facade_area'] = entity['perimeter'] * entity['ftf']
spatial_containment.aggregate(
    property='facade_area',
    label='subtotal_facade_area')

We now set up operational and capital costing parameters that are based off
either areas or volume, as well as growth rates:

In [29]:
floor_opex_per_area_pa = dict(
    office=-150,
    retail=-225,
    residential=-125,
    parking=-65,
    )
facade_opex_per_area_pa = dict(
    office=-50,
    retail=-100,
    residential=-50,
    parking=-25,
    )
opex_growth_rate = 0.035
floor_capex_per_area_pa = dict(
    office=-100,
    retail=-150,
    residential=-75,
    parking=-50,
    )
cores_capex_per_vol_pa = -100
plant_capex_per_vol_pa = -1000
capex_growth_rate = 0.035

Now we can set up our `Costs` class to produce cost `Flow`s and `Stream`s:

In [30]:
class Costs:
    def __init__(
            self,
            params: dict,
            spans: Spans):
        self.params = params
        self.spans = spans

In [31]:
@rk.update_class(Costs)
class Costs:
    def generate_space_costs(self):
        floor_opex = rk.flux.Flow.from_projection(
            name='Floor-derived Operating Expenses',
            value=self.params['initial_floor_opex'],
            proj=rk.projection.Extrapolation(
            form=rk.extrapolation.Compounding(
                rate=self.params['opex_growth_rate']),
            sequence=self.spans.calc_span.to_index(period_type=self.params['period_type'])),
            units=currency.units)
        facade_opex = rk.flux.Flow.from_projection(
            name='Facade-derived Operating Expenses',
            value=self.params['initial_facade_opex'],
            proj=rk.projection.Extrapolation(
            form=rk.extrapolation.Compounding(
                rate=self.params['opex_growth_rate']),
            sequence=self.spans.calc_span.to_index(period_type=self.params['period_type'])),
            units=currency.units)
        self.opex = rk.flux.Stream(
            name='Space Operating Expenses',
            flows=[floor_opex, facade_opex],
            period_type=self.params['period_type'])

        floor_capex = rk.flux.Flow.from_projection(
            name='Floor-derived Capital Expenses',
            value=self.params['initial_floor_capex'],
            proj=rk.projection.Extrapolation(
            form=rk.extrapolation.Compounding(
                rate=self.params['capex_growth_rate']),
            sequence=self.spans.calc_span.to_index(period_type=rk.periodicity.Type.SEMIDECADE)),
            units=currency.units)
        self.floor_capex = floor_capex.trim_to_span(self.spans.calc_span)  # This is to avoid issues with using >yearly periodicity in projection

We will split the generation of space-based costs from the utility-based costs,
as they will be applied to different `Entity` types:

In [32]:
@rk.update_class(Costs)
class Costs:
    def generate_utils_costs(self):
        utils_capex = rk.flux.Flow.from_projection(
            name='Utilities-derived Capital Expenses',
            value=self.params['initial_utility_capex'],
            proj=rk.projection.Extrapolation(
                form=rk.extrapolation.Compounding(
                    rate=self.params['capex_growth_rate']),
                sequence=self.spans.calc_span.to_index(period_type=rk.periodicity.Type.SEMIDECADE)),
            units=currency.units)
        self.utils_capex = utils_capex.trim_to_span(self.spans.calc_span)  # This is to avoid issues with using >yearly periodicity in projection

Now we can run the `Costs` class against 'floor' and 'utilities' `Entity`s:

In [33]:
for floor in floors:
    floor_params = params.copy()

    floor_params['initial_floor_opex'] = floor_opex_per_area_pa[floor['use']] * floor['subtotal_gfa']
    floor_params['initial_facade_opex'] = facade_opex_per_area_pa[floor['use']] * floor['subtotal_facade_area']
    floor_params['initial_floor_capex'] = floor_capex_per_area_pa[floor['use']] * floor['subtotal_gfa']
    floor_params['opex_growth_rate'] = opex_growth_rate
    floor_params['capex_growth_rate'] = capex_growth_rate
    floor['params'] = floor_params

    floor['events'] = {} if not hasattr(floor, 'events') else floor['events']

    costs = Costs(
        params=floor_params,
        spans=spans)
    costs.generate_space_costs()

    floor['opex'] = costs.opex
    floor['capex'] = costs.floor_capex

The utilities are a bit different, as (for this example), their capex is a
factor of their volume. We can see that the utilities are the cores and plant
of the property:

In [34]:
utilities = property.filter_by_type(entity_type='utilities', is_assembly=False)

In [35]:
key = 'type'
isolated_ids = [entity['id'] for entity in utilities.get_entities().values()]

In [36]:
url = 'https://speckle.xyz/streams/{0}/commits/{1}?filter={2}'.format(
    stream_id,
    commit_id,
    urllib.parse.quote(json.dumps(
        dict(
            propertyInfoKey = key,
            isolatedIds = isolated_ids))))
IFrame(url, width='100%', height=800)

In [37]:
for utility in utilities.get_entities().values():
    utility_params = params.copy()

    if 'plant' in utility['name']:
        utility_params['initial_utility_capex'] = plant_capex_per_vol_pa * utility['volume']
    elif 'cores' in utility['name']:
        utility_params['initial_utility_capex'] = cores_capex_per_vol_pa * utility['volume']
    utility_params['capex_growth_rate'] = capex_growth_rate
    utility['params'] = utility_params

    utility['events'] = {} if not hasattr(utility, 'events') else utility['events']

    costs = Costs(
        params=utility_params,
        spans=spans)
    costs.generate_utils_costs()

    utility['capex'] = costs.utils_capex

In [38]:
list(utilities.get_entities().values())[0]['capex']

date,Utilities-derived Capital Expenses
2005-12-31 00:00:00,"-$1,083,323.23"
2010-12-31 00:00:00,"-$1,121,239.55"


And finally aggregate them:

In [39]:
def aggregate_opex(**kwargs):
    return rk.graph.Entity.aggregate_flows(
        **kwargs,
        name='Operational Expenses',
        period_type=period_type)
property.aggregate(
    function=aggregate_opex,
    property='opex',
    label='subtotal_opex',
    relationship_type='spatiallyContains')

In [40]:
def aggregate_capex(**kwargs):
    return rk.graph.Entity.aggregate_flows(
        **kwargs,
        name='Capital Expenses',
        period_type=period_type)
property.aggregate(
    function=aggregate_capex,
    property='capex',
    label='subtotal_capex',
    relationship_type='spatiallyContains')

If we wish to see the ultimate aggregation of those costs, we can do so by
totalling them at their root `Entity`:

In [41]:
property_opex = property.get_roots()['spatiallyContains'][0]['subtotal_opex'].sum()
property_capex = property.get_roots()['spatiallyContains'][0]['subtotal_capex'].sum()

In [42]:
print('Property Average Periodic OPEX: ${:,.2f}'.format(property_opex.movements.mean()))
print('Property Average Periodic CAPEX: ${:,.2f}'.format(property_capex.movements.mean()))

Property Average Periodic OPEX: $-7,655,727.08
Property Average Periodic CAPEX: $-2,134,823.79


To get a sense check if this is reasonable, we can compare the opex and capex
to the PGI. First let's aggregate the PGI for all `Entity`s:

In [43]:
def aggregate_pgis(**kwargs):
    return rk.graph.Entity.aggregate_flows(
        **kwargs,
        name='Potential Gross Income',
        period_type=period_type)

In [44]:
spatial_containment.aggregate(
    property='pgi',
    function=aggregate_pgis,
    label='subtotal_pgi')

And total it to the root `Entity`:

In [45]:
property_pgi = property.get_roots()['spatiallyContains'][0]['subtotal_pgi'].sum()
print('Property Average Periodic PGI: ${:,.2f}'.format(property_pgi.movements.mean()))

Property Average Periodic PGI: $20,973,469.58


Now we can calculate the Expenses ratios:

In [46]:
print('Property OpEx as proportion of PGI: {:.2%}'.format(-property_opex.movements.mean() / property_pgi.movements.mean()))

Property OpEx as proportion of PGI: 36.50%


In [47]:
print('Property CapEx as proportion of PGI: {:.2%}'.format(-property_capex.movements.mean() / property_pgi.movements.mean()))

Property CapEx as proportion of PGI: 10.18%


Looking at just the cost-producing `Entity`s, we can see that in the OpEx and
CapEx breakdowns, there is a significant amount of cost generated by non-floor
(i.e., non-revenue-generating) `Entity`s, like cores and plant. This type of
analysis is important in understanding the cost drivers of a property, and
where the most effective cost-saving measures could be applied.

In [48]:
opex = spatial_containment.sunburst(property='subtotal_opex')
capex = spatial_containment.sunburst(property='subtotal_capex')
fig = plotly.subplots.make_subplots(
    rows=2, cols=1,
    specs=[[{"type": "sunburst"}], [{"type": "sunburst"}]],
    subplot_titles=('Total Operational Expenses', 'Total Capital Expenses'))
fig.append_trace(opex, row=1, col=1)
fig.append_trace(capex, row=2, col=1)
filename='aggregate_exp_chart.html'

py.plot(fig, filename=f'assets/{filename}', auto_open=False)
IFrame(src=f'./{filename}', width='100%', height=800 * len(fig.data))

## Calculating Overall Valuation
Now that we have our revenues and cost line items (`Flow`s) generated per
`Entity`, we can accumulate them into Net Operating Income (NOI) and
Net Annual Cashflow (NACF) subtotals, as well as the reversion (disposition, or
sale) of the whole asset, in order to determine the overall valuation of the
property.

We can also calculate the net position of each `Entity` as its 'Net Annual
Cashflow (NACF)':

In [49]:
for entity in property.get_entities().values():
    noi = [getattr(entity, 'egi', None), getattr(entity, 'opex', None)]
    noi = [flow.sum() if isinstance(flow, rk.flux.Stream) else flow for flow in list(filter(None, noi))]
    if len(noi) > 0:
        entity['noi'] = rk.flux.Stream(
            name='Net Operating Income',
            flows=noi,
            period_type=period_type)
        capex = getattr(entity, 'capex', None)
        if capex is not None:
            entity['nacf'] = rk.flux.Stream(
                name='Net Annual Cashflow',
                flows=entity['noi'].flows + [capex],
                period_type=period_type)

In [50]:
def aggregate_noi(**kwargs):
    return rk.graph.Entity.aggregate_flows(
        **kwargs,
        name='Net Operating Income',
        period_type=period_type)
property.aggregate(
    function=aggregate_noi,
    property='noi',
    label='subtotal_noi',
    relationship_type='spatiallyContains')

In [51]:
def aggregate_nacf(**kwargs):
    return rk.graph.Entity.aggregate_flows(
        **kwargs,
        name='Net Annual Cashflow',
        period_type=period_type)
property.aggregate(
    function=aggregate_nacf,
    property='nacf',
    label='subtotal_nacf',
    relationship_type='spatiallyContains')

### Reversion
To calculate the reversion cashflow, we set up a period that spans the 10th year
of the project. This only applies to the asset as a whole, so it is recorded
at the root `Entity`:

In [52]:
root = property.get_roots()['spatiallyContains'][0]

In [53]:
reversion_span = rk.span.Span.from_num_periods(
    name='Reversion',
    date=params['start_date'] + pd.DateOffset(years=9),
    period_type=params['period_type'],
    num_periods=1)

exit_caprate = 0.05
reversion_flow = rk.flux.Flow.from_projection(
    name='Reversion',
    value=root['subtotal_nacf'].sum().movements.values[-1] / exit_caprate,
    proj=rk.projection.Distribution(
        form=rk.distribution.Uniform(),
        sequence=reversion_span.to_index(period_type=params['period_type'])),
    units=currency.units)
root['reversion'] = reversion_flow
reversion_flow

date,Reversion
2010-12-31 00:00:00,"$275,897,434.55"


Now we can aggregate the net and reversion cashflows in the root `Entity` in
order to derive the project's complete cashflows:

In [54]:
net_cashflows_with_reversion = rk.flux.Stream(
    name='Net Cashflow with Reversion',
    flows=[root['subtotal_nacf'].sum(), root['reversion']],
    period_type=rk.periodicity.Type.YEAR).trim_to_span(
    rk.span.Span(
        start_date=params['start_date'],
        end_date=reversion_span.end_date)
    )
root['nacf_reversion'] = net_cashflows_with_reversion
net_cashflows_with_reversion

date,Net Annual Cashflow for 4fd8cf16-d01f-4d1e-964b-87942232e857 [property] Aggregation (sum),Reversion
2001,"$8,894,347.95",0
2002,"$9,276,145.32",0
2003,"$9,678,373.39",0
2004,"$10,102,249.12",0
2005,"$6,932,792.13",0
2006,"$11,020,193.91",0
2007,"$11,517,093.98",0
2008,"$12,041,314.24",0
2009,"$12,594,500.24",0
2010,"$9,435,557.80","$275,897,434.55"


In [55]:
table = rk.flux.Stream(
    name='Table 1.1',
    flows=[
        root['subtotal_egi'].sum(),
        root['subtotal_opex'].sum(),
        root['subtotal_noi'].sum(),
        root['subtotal_capex'].sum(),
        root['subtotal_nacf'].sum(),
        root['reversion'],
        root['nacf_reversion'].sum()
        ],
    period_type=rk.periodicity.Type.YEAR
    )
table

date,Effective Gross Income for 4fd8cf16-d01f-4d1e-964b-87942232e857 [property] Aggregation (sum),Operational Expenses for 4fd8cf16-d01f-4d1e-964b-87942232e857 [property] Aggregation (sum),Net Operating Income for 4fd8cf16-d01f-4d1e-964b-87942232e857 [property] Aggregation (sum),Capital Expenses for 4fd8cf16-d01f-4d1e-964b-87942232e857 [property] Aggregation (sum),Net Annual Cashflow for 4fd8cf16-d01f-4d1e-964b-87942232e857 [property] Aggregation (sum),Reversion,Net Cashflow with Reversion (sum)
2001,"$15,302,280.51","-$6,407,932.55","$8,894,347.95",0,"$8,894,347.95",0,"$8,894,347.95"
2002,"$15,908,355.51","-$6,632,210.19","$9,276,145.32",0,"$9,276,145.32",0,"$9,276,145.32"
2003,"$16,542,710.94","-$6,864,337.55","$9,678,373.39",0,"$9,678,373.39",0,"$9,678,373.39"
2004,"$17,206,838.49","-$7,104,589.37","$10,102,249.12",0,"$10,102,249.12",0,"$10,102,249.12"
2005,"$17,902,315.02","-$7,353,249.99","$10,549,065.03","-$6,294,320.75","$6,932,792.13",0,"$6,932,792.13"
2006,"$18,630,807.66","-$7,610,613.74","$11,020,193.91",$0.00,"$11,020,193.91",0,"$11,020,193.91"
2007,"$19,394,079.21","-$7,876,985.22","$11,517,093.98",$0.00,"$11,517,093.98",0,"$11,517,093.98"
2008,"$20,193,993.94","-$8,152,679.71","$12,041,314.24",$0.00,"$12,041,314.24",0,"$12,041,314.24"
2009,"$21,032,523.74","-$8,438,023.50","$12,594,500.24",$0.00,"$12,594,500.24",0,"$12,594,500.24"
2010,"$21,911,754.57","-$8,733,354.32","$13,178,400.25","-$6,514,621.98","$9,435,557.80","$275,897,434.55","$285,332,992.35"


### Valuation
With our net cashflows including reversion, if we specify a discount rate, we
can calculate the present value of the property:

In [56]:
discount_rate = 0.07
pvs = root['nacf_reversion'].sum().pv(
    name='Present Value',
    period_type=rk.periodicity.Type.YEAR,
    discount_rate=discount_rate)
pvs

date,Present Value
2001-12-31 00:00:00,"$8,312,474.72"
2002-12-31 00:00:00,"$8,102,144.57"
2003-12-31 00:00:00,"$7,900,435.65"
2004-12-31 00:00:00,"$7,706,957.49"
2005-12-31 00:00:00,"$4,942,984.97"
2006-12-31 00:00:00,"$7,343,220.52"
2007-12-31 00:00:00,"$7,172,267.30"
2008-12-31 00:00:00,"$7,008,154.52"
2009-12-31 00:00:00,"$6,850,573.65"
2010-12-31 00:00:00,"$145,048,824.68"


In [57]:
property_pv = pvs.collapse().movements.item()
print('Property PV: ${:,.0f}'.format(property_pv))

Property PV: $210,388,038
